In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint

In [4]:
import constants
from core.tools.metrics import *
from core.models.stat_models import *
from core.models.baseline_rnn import *
from core.tools.visualize import *
from core.tools.time_series import *
from core.tools.data_import import *
import core.models.baseline_lstm as baseline_lstm

In [5]:
# Avaiable datasets, loaded from constants.
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")
FILE_DIR = constants.DATA_DIR[choice]

{'0': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 '1': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 '2': '/home/ubuntu/AnnEconForecast/data/UNRATE.csv',
 '3': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> 0


In [6]:
print(f"Dataset chosen: \n{FILE_DIR}")

Dataset chosen: 
/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv


In [7]:
# Pre-processing Parameters
PERIODS = 1
ORDER = 1
LAGS = 12
TRAIN_RATIO = 0.8

In [8]:
# Neural Network Parameters
NUM_TIME_STEPS = LAGS
# Number of series used to predict. (including concurrent)
NUM_INPUTS = 1
NUM_OUTPUTS = 1
NUM_NEURONS = 64
# Number of output series
LEARNING_RATE = 0.01
EPOCHS = 50
# Training Settings
REPORT_PERIODS = 1
TENSORBOARD_DIR = "./tensorboard/test/1"

In [9]:
parameters = {
    "epochs": 50,
    "num_time_steps": LAGS,
    "num_inputs": 1,
    "num_outputs": 1,
    "num_neurons": 64,
    "learning_rate": 0.01,
    "report_periods": 1,
    "tensorboard_dir": "./tensorboard/test/1"
}

In [10]:
prepared_df = baseline_lstm.prepare_dataset(
    file_dir=FILE_DIR,
    periods=PERIODS,
    order=ORDER
)

Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
First few rows of dataset loaded:
            UNRATE_period1_order1
DATE                             
1948-02-01                    0.4
1948-03-01                    0.2
1948-04-01                   -0.1
1948-05-01                   -0.4
1948-06-01                    0.1


In [11]:
# Normalize data
(X_train, X_val, X_test,
 y_train, y_val, y_test) = baseline_lstm.normalize(
    df=prepared_df,
    train_ratio=TRAIN_RATIO,
    lags=LAGS
)

Total 836 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (836, 12, 1), y shape = (836, 1, 1)
Training and testing set generated,        
X_train shape: (601, 12, 1)        
y_train shape: (601, 1)        
X_test shape: (168, 12, 1)        
y_test shape: (168, 1)        
X_validation shape: (67, 12, 1)        
y_validation shape: (67, 1)


In [12]:
data_collection = {
    "X_train": X_train,
    "X_val": X_val,
    "X_test": X_test,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [13]:
# Run the model
baseline_lstm.exec_core(
    parameters=parameters,
    data_collection=data_collection,
    clip_grad=None
)

Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.

Iteration [0], Training MSE 5.4440751; Validation MSE 1.4397573

Iteration [1], Training MSE 2.4903817; Validation MSE 1.0182823

Iteration [2], Training MSE 1.6175922; Validation MSE 0.6213005

Iteration [3], Training MSE 1.7269717; Validation MSE 0.6327033

Iteration [4], Training MSE 1.8405190; Validation MSE 0.9295890

Iteration [5], Training MSE 1.8421853; Validation MSE 1.0777681

Iteration [6], Training MSE 1.5592966; Validation MSE 0.8378349

Iteration [7], Training MSE 1.1745561; Validation MSE 0.4896498

Iteration [8], Training MSE 0.9951801; Validation MSE 0.3500825

Iteration [9], Training MSE 1.0867471; Validation MSE 0.4658414

Iteration [10], Training MSE 1.2418327; Validation MSE 0.6260558

Iteration [11], Training MSE 1.2537372; Validation MSE 0.6297064

Iteration [12], Training MSE 1.1372035; Validation M

{'mae': 0.5540482, 'mse': 0.4814217, 'rmse': 0.69384557, 'mape': 4.4459753}